In [118]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [119]:
pip install google-api-python-client

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [120]:
from __future__ import print_function
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import datetime

In [121]:
# If modifying these scopes, delete the file token_sheets.json.
SCOPES_SHEETS = ['https://www.googleapis.com/auth/spreadsheets']

# https://docs.google.com/spreadsheets/d/1is8ZmX5mnwDU0YESiHi_yRqhysTJRF0UePIVNkzAOhk/edit#gid=487175113 sheets do cefacil

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1is8ZmX5mnwDU0YESiHi_yRqhysTJRF0UePIVNkzAOhk'
SAMPLE_RANGE_NAME = 'Reservas autorizadas!A:H'


creds_sheets = None
# The file token_sheets.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token_sheets.json'):
    creds_sheets = Credentials.from_authorized_user_file('token_sheets.json', SCOPES_SHEETS)
# If there are no (valid) credentials available, let the user log in.
if not creds_sheets or not creds_sheets.valid:
    if creds_sheets and creds_sheets.expired and creds_sheets.refresh_token:
        creds_sheets.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES_SHEETS)
        creds_sheets = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token_sheets.json', 'w') as token:
        token.write(creds_sheets.to_json())

In [122]:
SCOPES_CALENDAR = ['https://www.googleapis.com/auth/calendar']

creds_calendar = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token_calendar.json'):
    creds_calendar = Credentials.from_authorized_user_file('token_calendar.json', SCOPES_CALENDAR)
# If there are no (valid) credentials available, let the user log in.
if not creds_calendar or not creds_calendar.valid:
    if creds_calendar and creds_calendar.expired and creds_calendar.refresh_token:
        creds_calendar.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES_CALENDAR)
        creds_calendar = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token_calendar.json', 'w') as token:
        token.write(creds_calendar.to_json())

In [123]:
def get_events_from_sheets(spreadsheet_id, range_name, creds):

    service = build('sheets', 'v4', credentials=creds)
    # Make the request to get the values
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name, majorDimension='ROWS').execute()

    events = []
    for i in result['values']:
        if(not i == []):
            events.append(i)
    
    return events[1:]

In [124]:
def show_events_sheet(spreadsheet_id, range_name,creds):
    
    eventos = get_events_from_sheets(spreadsheet_id, range_name, creds)

    for i in eventos:

        print(i)

        # Get the last row
        #last_row = result['values'][-1]
        last_row = i

        # Extract the column values
        local = last_row[0]
        start_date = last_row[1]
        start_time = last_row[2]
        end_time = last_row[3]
        name = last_row[4]
        n_usp = last_row[5]
        email = last_row[6]
        description = last_row[7]

        # Print the values
        print("Date:", start_date)
        print("Time:", start_time)
        print("Name:", name)
        print("ID Number:", n_usp)
        print("Email:", email)
        print("Local:", local)
        print("Start Date:", start_date)
        print("Start Time:", start_time)
        print("End Time:", end_time)
        print("Description:", description)
        print()

In [125]:
def week_events(creds, calendar_id):

    try:
        service = build('calendar', 'v3', credentials=creds)

        # Call the Calendar API
        now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
        one_week_later = (datetime.datetime.utcnow() + datetime.timedelta(weeks=1)).isoformat() + 'Z'   # Add one week to the current tim
    
        events_result = service.events().list(calendarId=calendar_id, timeMin=now, timeMax=one_week_later,
                                                singleEvents=True, orderBy='startTime').execute()
        events = events_result.get('items', [])

        if not events:
            print('No upcoming events found.')
            return []
        else:
           return events

    except HttpError as error:
        print('An error occurred: %s' % error)

In [126]:
def create_event(id_agenda, titulo, descricao, local, data_inicio, hora_inicio, data_fim, hora_fim, creds, convidados=None):
    
    service = build('calendar', 'v3', credentials=creds)
    try:

        evento = {
            'summary': titulo,
            'description': descricao,
            'location': local,
            'start': {
                'dateTime': f'{data_inicio}T{hora_inicio}',
                'timeZone': 'America/Sao_Paulo',  # Substitua pelo fuso horário correto
            },
            'end': {
                'dateTime': f'{data_fim}T{hora_fim}',
                'timeZone': 'America/Sao_Paulo',  # Substitua pelo fuso horário correto
            },
            'reminders': {
                'useDefault': False,
                'overrides': [
                    {'method': 'email', 'minutes': 60}
                ],
            },
            'guestsCanInviteOthers': True,
            'guestsCanSeeOtherGuests': True,
        }

        if convidados is not None:
            evento['attendees'] = [{'email': email} for email in convidados]

        evento_criado = service.events().insert(calendarId=id_agenda, body=evento).execute()
        print("Evento criado:")
        print(f"Título: {evento_criado['summary']}")
        print(f"Início: {evento_criado['start']['dateTime']}")
        print(f"Fim: {evento_criado['end']['dateTime']}")

    except HttpError as error:
        print('An error occurred: %s' % error)

In [127]:
def convert_date_format(date_string):
    day, month, year = date_string.split('/')
    new_date_string = f"{year}-{month}-{day}"
    return new_date_string

def convert_event_sheet_to_event_calendar(event):

    local = event[0]
    start_date = convert_date_format(event[1])
    start_time = event[2] + '-03:00'
    end_time = event[3] + '-03:00'
    name = event[4]
    n_usp = event[5]
    email = event[6]
    description = event[7]

    return local, start_date, start_time, end_time, name, n_usp, email, description

In [128]:
def create_week_events(spreadsheet_id, range_name, creds_s, id_agenda, creds_c):

    events_from_sheets_list = get_events_from_sheets(spreadsheet_id, range_name, creds_s)
    events_from_calendar = week_events(creds_c, id_agenda)

    # pegar eventos da agenda da semana
    for event_sheet in events_from_sheets_list:
        
        local, start_date, start_time, end_time, name, n_usp, email, description = convert_event_sheet_to_event_calendar(event_sheet)
        titulo = 'Reserva ' + local

        found = False

        if len(events_from_calendar) > 0:
            for event_calendar in events_from_calendar:
                if event_calendar['start']['dateTime'] == f'{start_date}T{start_time}' and \
                    event_calendar['end']['dateTime'] == f'{start_date}T{end_time}' and \
                        event_calendar['location'] == local:
                    found = True
                    break

        if not found:
           create_event(id_agenda, titulo, description, local, start_date, start_time, start_date, end_time, creds_calendar, [email])

create_week_events(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, creds_sheets, 'testecefacil@gmail.com', creds_calendar)